In [ ]:
# This file creates dataframe from json dataset files and inserts data into mysql server.
# This was the first method using mysql connector which is not very efficient so it takes considerable time to run

In [2]:
!pip3 install pandas
!pip3 install SQLAlchemy
!pip3 install PyMySQL
import pymysql
from urllib.parse import quote 
import mysql.connector
import sqlalchemy
from sqlalchemy.engine import create_engine
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
#connect to mysql servel
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Parvashah@123",
  database="yelpdb"
)

print(mydb)

mycursor = mydb.cursor()


In [4]:
# create business table
mycursor.execute("DROP TABLE IF EXISTS business")
mycursor.execute("CREATE TABLE business (business_id VARCHAR(255) PRIMARY KEY, name VARCHAR(255), address VARCHAR(255), city VARCHAR(255), state TEXT(2), postal_code INT(5), stars DECIMAL(10,1), review_count INT, is_open BOOL)")

In [5]:
# Insert data into business table from json file

# dataframe creation
df_b = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
# delete unwanted columns
del df_b['attributes'], df_b['hours'], df_b['categories'], df_b['latitude'], df_b['longitude']

sql = "INSERT IGNORE INTO business (business_id, name, address, city, state, postal_code, stars, review_count, is_open) VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s)"
val = list(df_b.itertuples(index=False,name=None))

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

150346 was inserted.


In [28]:
# create user table
mycursor.execute("DROP TABLE IF EXISTS user")
mycursor.execute("CREATE TABLE user (user_id VARCHAR(255) PRIMARY KEY, name VARCHAR(255), review_count INT, average_stars FLOAT, yelping_since DATETIME)")

In [32]:
chunks = pd.read_json('yelp_dataset/yelp_academic_dataset_user.json', lines=True, chunksize = 10000)
users = pd.DataFrame()
for chunk in chunks:
    users = pd.concat([users, chunk])

del users['useful'], users['funny'], users['cool'], users['elite'], users['friends'], users['fans'], users['compliment_hot'], users['compliment_more'], users['compliment_profile'], users['compliment_cute'], users['compliment_list'], users['compliment_note'], users['compliment_plain'], users['compliment_cool'], users['compliment_funny'], users['compliment_writer'], users['compliment_photos']
val = list(users.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO user (user_id, name, review_count, average_stars, yelping_since) VALUES (%s, %s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")

1987897 was inserted.


In [1]:
# create review table
mycursor.execute("DROP TABLE IF EXISTS review")
mycursor.execute("CREATE TABLE review (review_id VARCHAR(255) PRIMARY KEY, user_id VARCHAR(255), business_id VARCHAR(255), stars INT(1), text TEXT(65535), date DATETIME, CONSTRAINT user_id_fk FOREIGN KEY (user_id) REFERENCES user (user_id) ON UPDATE CASCADE ON DELETE CASCADE,CONSTRAINT business_id_fk FOREIGN KEY (business_id) REFERENCES business (business_id) ON UPDATE CASCADE ON DELETE CASCADE)")

NameError: name 'mycursor' is not defined

In [36]:
# Insert data into review table from json file
# dataframe creation is done in chunks since it is a large file which cannot be read at once.
chunks = pd.read_json('yelp_dataset/yelp_academic_dataset_review.json', lines=True, chunksize = 10000)
reviews = pd.DataFrame()
for chunk in chunks:
    reviews = pd.concat([reviews, chunk])
# delete unwanted columns from dataframe
del reviews['useful'], reviews['funny'], reviews['cool']


KeyboardInterrupt: 

In [37]:
# Insert the dataframe in to sql
val = list(reviews.itertuples(index=False,name=None))
sql = "INSERT IGNORE INTO review (review_id, user_id, business_id, stars, text, date) VALUES (%s, %s, %s, %s, %s, %s)"

mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "was inserted.")


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
5089995,1sb3k7AVkPuHRh4NOLpICg,RVXG9BjSqAEvPAzUAsY_8g,mpeLyxsDTm5RqZkFT--SdQ,1,0,0,0,The table was dirty when we sat down. The drin...,2017-08-13 03:32:30
5089996,dQ22aY7R_TLhltogeAY3ZQ,ZDc-keECogZyw0U2x6xYpw,SjiRaZoWaDpkCCpq5SsrMw,5,2,1,2,We stopped in for lunch and sat at the bar wit...,2014-07-21 17:53:09
5089997,UEcL3XitFeg-wtZE5E_6Tg,vGifaZpcscr0WOOp2ctadw,_6woRce_3sGy5zDrpbM4Vg,5,0,0,0,Went here to get a basic service on my car. S...,2016-06-27 19:27:51
5089998,i6keVbnvkO5MZoCdAdR_8w,qTMK2qr6ngof4fe29qyooA,9U5nEjCeHJ1yr8pEYkKB1A,2,2,4,1,"I'm not a fan of the Gargoyle, plain and simpl...",2011-02-08 21:27:51
